In [2]:
from numpy import *
import csv
import numpy
import pandas as pd
import math
import os

Crispr_data='/Users/jiahao/Desktop/crisprCas9/SSL/data/Crispr_Data.csv'
Crispr_data2= '/Users/jiahao/Desktop/crisprCas9/SSL/equal/data/Crispr_Data2.txt'
Crispr_positive='/Users/jiahao/Desktop/crisprCas9/SSL/equal/data/Crispr_SSL_positive.txt'
Crispr_negative='/Users/jiahao/Desktop/crisprCas9/SSL/equal/data/Crispr_SSL_negative'


Name_data_list = ['cfdScore','MITwebsite','MITscores','CropitScores','CCTopSores','PhastCons','phylop','Promoter',
                  'Enhancer','segwayPromoter','segwayEnhancer']
Distance_path='/Users/jiahao/Desktop/crisprCas9/SSL/equal/distance/'

Result = '/Users/jiahao/Desktop/crisprCas9/SSL/equal/result/'
input_list=[2,3,4,5,6,7,8,9,10,11,12]

In [3]:
input_list=[2,3,4,5,6,7,8,9,10,11,12]
feature_range=[]
#记录每个特征的范围
raw_num=0
for i in range(0,11):
    
    min_value=10000
    max_value=-10000
    csvfile = file(Crispr_data2,'rb') #读文件
    reader = csv.reader(csvfile)
    for row in reader:
        if i==0:
            raw_num=raw_num+1
        if row[input_list[i]]=='NA':
            current = 0
        else:
            current=float(row[input_list[i]])
        if current > max_value:
            max_value = current
        if current < min_value:
            min_value = current
    csvfile.close()
    
    feature_range.append(array([min_value,max_value,max_value-min_value]))

    print Name_data_list[i], feature_range[i][0],'~',feature_range[i][1],'~',feature_range[i][2]

print "count=",raw_num

cfdScore 0.0 ~ 1.0 ~ 1.0
MITwebsite 0.0 ~ 100.0 ~ 100.0
MITscores 0.001756701 ~ 100.0 ~ 99.998243299
CropitScores 230.0 ~ 642.5 ~ 412.5
CCTopSores 104.9049558 ~ 222.8 ~ 117.89504420000002
PhastCons 0.0 ~ 1.0 ~ 1.0
phylop -2.608695652 ~ 7.304347826 ~ 9.913043478
Promoter 0.0 ~ 1.0 ~ 1.0
Enhancer 0.0 ~ 2.0 ~ 2.0
segwayPromoter 0.0 ~ 1.0 ~ 1.0
segwayEnhancer 0.0 ~ 1.0 ~ 1.0
count= 25332


In [18]:
for i in range(0,5):
    
#     border_positive_max=feature_range[i][0]
#     border_positive_min=feature_range[i][1]
#     border_negative_max=feature_range[i][0]
#     border_negative_min=feature_range[i][1]
    border_positive_max=-100
    border_positive_min=900
    border_negative_max=-100
    border_negative_min=900
    csvfile = file(Crispr_data2,'rb') #读文件
    reader = csv.reader(csvfile)
    for row in reader:
        if row[input_list[i]]=='NA':
            current = 0
        else:
            current=float(row[input_list[i]])
        if (row[1]=='1') and (current > border_positive_max):
            border_positive_max = current
            
        if (row[1]=='1' and current < border_positive_min):
            border_positive_min = current
            
        if (row[1]=='0' and current > border_negative_max):
            border_negative_max = current
            
        if (row[1]=='0' and current < border_negative_min):
            border_negative_min = current
    csvfile.close()

    print Name_data_list[i], 'p:',border_positive_min,'~',border_positive_max,'n:',border_negative_min,'~',border_negative_max
    print '       ',feature_range[i][0],' ',feature_range[i][1]

print "count=",raw_num

 cfdScore p: 0.0 ~ 1.0 n: 0.0 ~ 0.941176471
        0.0   1.0
MITwebsite p: 0.0 ~ 100.0 n: 0.0 ~ 7.414830508
        0.0   100.0
MITscores p: 0.023413405 ~ 100.0 n: 0.001756701 ~ 100.0
        0.001756701   100.0
CropitScores p: 387.5 ~ 642.5 n: 230.0 ~ 642.5
        230.0   642.5
CCTopSores p: 124.1050829 ~ 222.8 n: 104.9049558 ~ 222.8
        104.9049558   222.8
count= 25332


In [21]:
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score
from scipy.optimize import fsolve 

In [29]:
for index in range(80,90):
    dist_train = [[],[],[],[],[],[],[],[],[],[],[]] #dist_train为各特征权重矩阵W
    for i in range(0,11):
        #read input data
        filename=Distance_path+str(index)+'/Distance_feature'+str(i)+'.txt'
        csvfile = file(filename,'rb') #读文件，路径为各特征的距离矩阵文件
        reader_crispr = csv.reader(csvfile)
        for row in reader_crispr:
            dist_train[i].append(row) 
        csvfile.close()
    num=len(row)
        
    L_list = [[],[],[],[],[],[],[],[],[],[],[]]
    for i in range(0,11):
        Index = mat(numpy.eye(num))
        D = mat(numpy.eye(num))
        for i1 in range(0,num):
            d = 0.0
            for j1 in range(0,num):
                d = d+float(dist_train[i][i1][j1])
            D[i1,i1] = d
        L = D-mat(dist_train[i]).astype('float64')
        L_list[i] = L
        

    ##Read  Outcome Data
    o_true = []
    filename=Distance_path+str(index)+'/Crispr_Data'+str(index)+'.txt'
    csvfile = file(filename,'rb') #读文件，路径为各特征的距离矩阵文件
    reader = csv.reader(csvfile)
    for row in reader:
        o_true.append([float(row[1])])
    csvfile.close()


    l_5fold=[]
    l = range(0,num)
    l_train,l_test = train_test_split(l,test_size=0.2)
    l_5fold.append(l_test)
    l0_train,l0_test = train_test_split(l_train,test_size=0.5)
    l1_train,l1_test = train_test_split(l0_train,test_size=0.5)
    l2_train,l2_test = train_test_split(l0_test,test_size=0.5)
    l_5fold.append(l1_train)
    l_5fold.append(l1_test)
    l_5fold.append(l2_train)
    l_5fold.append(l2_test)

    
    directory_name=Result+str(index)
    if not os.path.exists(directory_name):
        os.mkdir(directory_name)
        
    for Data_flag in range(0,11):  #循环数据类型
        filename=directory_name+'/result_feature'+str(Data_flag)+'.txt'
        wcsvfile = file(filename,'wb') 
        writer = csv.writer(wcsvfile)
        max_auc = 0 #最佳AUC值
        best_u = 0

        for i in range(0,5):
            #5折交叉分类，80%数据作为训练数据，20%数据作为测试数据，测试数据无标签
            l_test = l_5fold[i]
            l_train = []
            for j in range(0,5):
                if i==j:
                    continue
                else:
                    for k in range(0,len(l_5fold[j])):
                        l_train.append(l_5fold[j][k] )

            o_train = []
            #根据训练集和测试集，分类训练和测试数据，生成o_train
            for j in range(0,num):
                if j in l_train:
                    o_train.append(o_true[j])
                elif j in l_test:
                    o_train.append([0.0])
            
            auc=-1
            u=0
            count=0
            #print 'Index','=',Index
            #print 'L_list=',L_list
            #print 'o_train=',o_train
            for u0 in numpy.arange(0,1.002,0.002):
                f1= Index+u0*L_list[Data_flag]
                f = mat(f1.I)*mat(o_train)
                tmp=roc_auc_score(o_true,f)
                #print 'u0=',u0,' auc=',tmp
                #print 'o_true=',o_true
                #print 'f=',f
                if (auc<tmp):
                    auc=tmp
                    u=u0
            
            best_u +=u
            max_auc +=auc
            

#             u = fsolve(func,0.5) #通过fsolve函数求解最佳参数u
#             auc=func(u)+1
#             best_u += u
#             max_auc += auc
            writer.writerow([round(u,6),round(auc,6)])
        
        #writer.writerow(['best_u','best_auc'])
        writer.writerow([round(best_u/5,6),round(max_auc/5,6)])
        wcsvfile.close()
        #print 'Off-target','-',Name_data_list[Data_flag],'max_auc:',round(max_auc/5,6) #得平均值AUC
        #print 'best_u:',round(best_u/5,6) #得平均值AUC


In [20]:
import csv
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
Name_data_list = ['CFDscore','MITwebsite','MITscores','CropitScores','CCTopSores','PhastCons','Phylop','Promoter',
                  'Enhancer','segwayPromoter','segwayEnhancer']
index=0;
Distance_path='/Users/jiahao/Desktop/crisprCas9/SSL/equal/distance/'
filename=Distance_path+str(index)+'/Crispr_Data'+str(index)+'.txt'
o_true = []
csvfile = file(filename,'rb') 
reader = csv.reader(csvfile)
for row in reader:
    o_true.append([float(row[1])])
csvfile.close()
plt.figure(figsize=(8, 8)) 
for i in range(0,11):
    #read input data
    csvfile = file(filename,'rb') 
    reader_crispr = csv.reader(csvfile)
    feature=[]
    for row in reader_crispr:
        feature.append(float(row[i+2])) 
    csvfile.close()
    num=len(feature)
    #auc=roc_auc_score(o_true,feature)
    fpr, tpr, thresholds = roc_curve(o_true, feature)
    roc_auc = auc(fpr, tpr)
    print Name_data_list[i],round(roc_auc,3)
    plt.plot(fpr, tpr, lw=1, alpha=0.3, label='%s (AUC = %0.4f)' % (Name_data_list[i],roc_auc))

plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC based on single feature')
plt.legend(loc="lower right")
outfname = "single feature.png"
plt.savefig(outfname, bbox_inches='tight')

CFDscore 0.916
MITwebsite 0.726
MITscores 0.892
CropitScores 0.831
CCTopSores 0.78
PhastCons 0.438
Phylop 0.49
Promoter 0.51
Enhancer 0.507
segwayPromoter 0.51
segwayEnhancer 0.513


In [50]:
#histogram.py
index=0;
Distance_path='/Users/jiahao/Desktop/crisprCas9/SSL/equal/record/'
filename=Distance_path+'rank'+'.txt'
csvfile = file(filename,'rb') 
reader = csv.reader(csvfile)
count=0
auc=[]
for row in reader:
    count+=1
    auc.append(float(row[1]))
csvfile.close()
print count
print auc

142
[0.919, 0.608, 0.869, 0.781, 0.748, 0.562, 0.528, 0.587, 0.605, 0.578, 0.544, 0.928, 0.946, 0.924, 0.919, 0.901, 0.914, 0.939, 0.966, 0.908, 0.916, 0.863, 0.811, 0.777, 0.659, 0.642, 0.859, 0.822, 0.589, 0.752, 0.892, 0.881, 0.888, 0.893, 0.922, 0.923, 0.889, 0.914, 0.814, 0.806, 0.815, 0.887, 0.849, 0.808, 0.844, 0.783, 0.785, 0.901, 0.891, 0.795, 0.833, 0.57, 0.806, 0.786, 0.565, 0.769, 0.836, 0.779, 0.55, 0.782, 0.853, 0.831, 0.836, 0.703, 0.823, 0.795, 0.951, 0.938, 0.939, 0.94, 0.937, 0.949, 0.956, 0.934, 0.938, 0.95, 0.948, 0.954, 0.951, 0.959, 0.962, 0.949, 0.945, 0.926, 0.934, 0.93, 0.943, 0.951, 0.93, 0.93, 0.929, 0.926, 0.937, 0.946, 0.925, 0.932, 0.886, 0.877, 0.886, 0.889, 0.928, 0.923, 0.888, 0.913, 0.833, 0.814, 0.818, 0.886, 0.863, 0.81, 0.895, 0.784, 0.777, 0.872, 0.853, 0.775, 0.898, 0.876, 0.89, 0.89, 0.923, 0.923, 0.892, 0.911, 0.884, 0.892, 0.927, 0.923, 0.89, 0.916, 0.835, 0.83, 0.877, 0.873, 0.828, 0.893, 0.941, 0.941, 0.929, 0.943, 0.879, 0.954]


In [56]:
import math
import random
from pandas import *
k=21
sort_index = argsort(auc)[:-(k+1):-1]
auc_sort=[]
print sort_index[0:k]
for i in range(k):
    auc_sort.append(auc[sort_index[i]])
print auc_sort

[ 0  1  2  3  4  7  5  6  8  9 10 11 12 13 14 15 16 18 17 19 21]
[0.962, 0.959, 0.956, 0.954, 0.954, 0.951, 0.951, 0.951, 0.95, 0.949, 0.949, 0.948, 0.946, 0.946, 0.945, 0.943, 0.943, 0.941, 0.941, 0.94, 0.939]


In [57]:
auc=auc_sort[1:]
print auc

[0.959, 0.956, 0.954, 0.954, 0.951, 0.951, 0.951, 0.95, 0.949, 0.949, 0.948, 0.946, 0.946, 0.945, 0.943, 0.943, 0.941, 0.941, 0.94, 0.939]


In [75]:
auc_adaboost=[0.862,0.875,0.742,0.903,0.921,0.905,0.851,0.938,0.914,0.891]

In [64]:
auc[:10]

[0.959, 0.956, 0.954, 0.954, 0.951, 0.951, 0.951, 0.95, 0.949, 0.949]

In [76]:
import pandas as pd

plt.figure(figsize=(8, 8)) 
# Generate data on commute times.
# size, scale = 1000, 10
bar_width = 0.55
#commutes = pd.Series(auc)
plt.figure(figsize=(1000,10))
fig, ax = plt.subplots()
ax.bar([1, 3, 5, 7, 9,11,13,15,17,19], auc[:10],bar_width,fc = '#607c8e', label='SSL')
ax.bar([2, 4, 6, 8, 10,12,14,16,18,20], auc_adaboost,bar_width,label='Adaboost')
#ax.bar(range(len(auc)), auc,  bar_width,fc = '#607c8e')
#commutes.plot.hist(grid=True, bins=20, rwidth=0.9,color='#607c8e')
index = np.arange(20)
#ax.set_xticks(index + bar_width / 2)
plt.title('Compare the performance of SSL and Adaboost')
plt.xlabel('Combinations of features')
plt.ylabel('AUC')
plt.legend(loc="lower right")
plt.grid(axis='y', alpha=0.75)
outfname = "histogram.png"
plt.savefig(outfname, bbox_inches='tight')
plt.close()